<a href="https://colab.research.google.com/github/Daeun-Danna-Lee/Object_Detection_Python/blob/main/Movie_Review_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 19.44 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install konlpy

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [3]:
import jpype

p = jpype.getDefaultJVMPath()
jpype.startJVM( p )

print(jpype.isJVMStarted())

True


In [ ]:
# %%bash
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# pip3 install /tmp/mecab-python-0.996

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.
Processing /tmp/mecab-python-0.996
  Created wheel for mecab-python: filename=mecab_python-0.996_ko_0.9.2-cp37-cp37m-linux_x86_64.whl size=141821 sha256=2fc4974526cdd4ffa383f5dc29df342872b59dae1496324ed420ba78b6983cc3
  Stored in directory: /root/.cache/pip/wheels/40/7b/9f/2922869bef86c3354ae7034f7a3647c573ee1997c2dad0290a
Failed to build mecab-python
  Attempting uninstall: mecab-python
    Found existing installation: mecab-python 0.996-ko-0.9.2
    Uninstalling mecab-python-0.996-ko-0.9.2:
      Successfully uninstalled mecab-python-0.996-ko-0.9.2
    Running setup.py install for mecab-python: started
    Running setup.py install for mecab-python: finished with status 'done'


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [25]:
import pandas as pd

train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
import nltk
nltk.download('all')

In [47]:
# 구둣점 제거 -> 공백으로 대체
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9]"," ")

In [48]:
train_df

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1


In [49]:
from nltk.tokenize import word_tokenize

train_text = []
test_text = []

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

for text in train_df['document'][:8000]:
  temp = word_tokenize(text)
  no_stopwords_ver = [word for word in temp if not word in stopwords]
  train_text.append(no_stopwords_ver)
for text in test_df['document'][8000:9000]:
  temp = word_tokenize(text)
  no_stopwords_ver = [word for word in temp if not word in stopwords]
  test_text.append(no_stopwords_ver)
train_label = train_df['label'][:8000]
test_label = test_df['label'][8000:9000]

In [50]:
train_text

[['아', '더빙', '진짜', '짜증나네요', '목소리'],
 ['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나'],
 ['너무재밓었다그래서보는것을추천한다'],
 ['교도소', '이야기구먼', '솔직히', '재미는', '없다', '평점', '조정'],
 ['사이몬페그의',
  '익살스런',
  '연기가',
  '돋보였던',
  '영화',
  '스파이더맨에서',
  '늙어보이기만',
  '했던',
  '커스틴',
  '던스트가',
  '너무나도',
  '이뻐보였다'],
 ['막', '걸음마', '뗀', '세부터', '초등학교', '학년생인', '살용영화', 'ㅋㅋㅋ', '별반개도', '아까움'],
 ['원작의', '긴장감을', '제대로', '살려내지못했다'],
 ['별',
  '반개도',
  '아깝다',
  '욕나온다',
  '이응경',
  '길용우',
  '연기생활이몇년인지',
  '정말',
  '발로해도',
  '그것보단',
  '낫겟다',
  '납치',
  '감금만반복반복',
  '이드라마는',
  '가족도없다',
  '연기못하는사람만모엿네'],
 ['액션이', '없는데도', '재미', '있는', '몇안되는', '영화'],
 ['왜케', '평점이', '낮은건데', '꽤', '볼만한데', '헐리우드식', '화려함에만', '너무', '길들여져', '있나'],
 ['걍인피니트가짱이다', '진짜짱이다'],
 ['볼때마다', '눈물나서', '죽겠다', '년대의', '향수자극', '허진호는', '감성절제멜로의', '달인이다'],
 ['울면서', '손들고', '횡단보도', '건널때', '뛰쳐나올뻔', '이범수', '연기', '드럽게못해'],
 ['담백하고',
  '깔끔해서',
  '좋다',
  '신문기사로만',
  '보다',
  '보면',
  '자꾸',
  '잊어버린다',
  '그들도',
  '사람이었다는',
  '것을'],
 ['취향은',
  '존중한다지만',
  '진짜',
  '내생에',
  '극장에서',
  '본',
  '영화중

In [ ]:
# # 토큰화

# from nltk.tokenize import word_tokenize
# from konlpy.tag import Kkma, Okt
# from konlpy.tag import Okt
# from tqdm import tqdm

# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train_text = []
# test_text = []

# okt = Okt()

# for text in tqdm(train_df['document']):
#   temp = okt.morphs("안녕하세요", stem=True)
#   no_stopwords_ver = [word for word in temp if not word in stopwords]
#   train_text.append(no_stopwords_ver)
# for text in tqdm(test_df['document']):
#   temp = okt.morphs(text, stem=True)
#   no_stopwords_ver = [word for word in temp if not word in stopwords]
#   test_text.append(no_stopwords_ver)
# train_label = train_df['label'][:8000]
# test_label = test_df['label'][8000:9000]

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def identity_tokenizer(text):
    return text

vetorizer = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False)    
x_train = vetorizer.fit_transform(train_text)
x_test = vetorizer.transform(test_text)

In [ ]:
data = data.toarray()

In [ ]:
data

In [14]:
feature_names = tfidf_vetorizer.get_feature_names()
print(feature_names)
print(data)

['!', '%', '&', "'", "''", "'드래곤'이란", "'라고", "'명작", "'신소율'씨", "'아", "'억지로", "'재현'해낸", "'하와이'에서의", '(', ')', '*', ',', ',기억에', ',영화', '-', '--', '-0-', '-ㅅ-', '.', '..', '...', '..ㅜㅜㅜㅜ', '..네이버', '..지나도그때의', './///', '.1점도아까움', '.ㅠ', '.그러나', '.나도', '.노답임', '.디카프리오', '.암튼', '.어차피', '.요즘', '.요즘은', '.울지', '.이', '.이딴영화', '.재미있음', '.전혀', '.정말노답', '.초딩들만', '.특히', '/./', '0.2정도', '0.5깎음', '007', '0점은', '1', '100분짜리', '100점', '10~20대', '10년', '10년동안에도', '10년이', '10억은', '10자이상', '10점', '10점씩', '10점으로도', '10점이다', '10점준', '10점줌', '12345678910', '13세', '13점~', '15년전', '15분짜리', '15세관람가라고', '18', '18..', '1914년작', '1982년', '1988년도', '1995년은', '1999년작', '19세', '1년', '1도', '1등', '1백만', '1백만을', '1빠의힘이', '1위', '1인', '1점', '1점도', '1점도아깝다', '1점주고', '1점주기도아까운', '1점줍니다', '1주기도', '1톤ㅋㅋ', '1편', '1편보다', '1편보단', '1편은', '1편을', '2,4인데..같은', '2.5', '2008년도에', '2010년에', '20대', '20대와', '24', '26년이', '26세', '2개', '2년만에', '2보다', '2시간', '2시간이', '2억5천인데', '2점', '2탄', '2탄까지', '2편', '2편에', '2편에서', '2편은', '2편을', '2편이', '2회하

NameError: ignored

In [52]:
# x_train = train_text
# x_test = test_text
y_train = train_label
y_test = test_label

In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
clf = MultinomialNB(alpha=.01)
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))


F1 = 0.7279118434372737
accuracy = 0.728
